In [1]:
import torch
from cot_probing.typing import *
import random
from transformers import AutoModelForCausalLM, AutoTokenizer, AwqConfig
from beartype import beartype

# model_name = "google/gemma-2-9b"
# model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cuda")
model_name = "hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4"
quantization_config = AwqConfig(
    bits=4,
    fuse_max_seq_len=1024,  # Note: Update this as per your use-case
    do_fuse=True,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="cuda",
    quantization_config=quantization_config,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/paperspace/cot-probing/.venv/lib/python3.10/site-packages/transformers/quantizers/auto.py:182: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['version', 'fuse_max_seq_len', 'exllama_config', 'modules_to_fuse', 'do_fuse']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [2]:
def load_and_split_file(file_path: str) -> list[str]:
    """
    Loads a text file and splits it by double line break.

    Args:
        file_path (str): The path to the text file.

    Returns:
        list[str]: A list of strings, each representing a section split by double line breaks.
    """
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()

    return content.split("\n\n")

In [36]:
qs_unb = load_and_split_file("diverse_unbiased.txt")
qs_yes = load_and_split_file("diverse_yes.txt")
qs_no = load_and_split_file("diverse_no.txt")

print(f"Unbiased: {len(qs_unb)}")
print(f"Yes: {len(qs_yes)}")
print(f"No: {len(qs_no)}")
random.seed(1)
shuffled_indices = random.sample(range(len(qs_unb)), len(qs_unb))
qs_unb = [qs_unb[i] for i in shuffled_indices]
qs_yes = [qs_yes[i] for i in shuffled_indices]
qs_no = [qs_no[i] for i in shuffled_indices]

Unbiased: 10
Yes: 10
No: 10


In [37]:
unb_fsps = "\n\n".join(qs_unb[1:])
yes_fsps = "\n\n".join(qs_yes[1:])
no_fsps = "\n\n".join(qs_no[1:])

split_string = "Reasoning:\n-"
q_yes = qs_yes[0].split(split_string)[0] + split_string
q_no = qs_no[0].split(split_string)[0] + split_string

In [38]:
print(q_yes)
print(q_no)

Question: Is one week from March 25, 2024 04/01/2024 in MM/DD/YYYY notation?
Reasoning:
-
Question: Is one week from March 25, 2024 03/31/2024 in MM/DD/YYYY notation?
Reasoning:
-


In [39]:
# Create all combinations of FSPs and questions
combinations = [
    (unb_fsps, q_yes, "unb_yes"),
    (no_fsps, q_yes, "no_yes"),
    (unb_fsps, q_no, "unb_no"),
    # (yes_fsps, q_yes, "yes_yes"),
    (yes_fsps, q_no, "yes_no"),
    # (no_fsps, q_no, "no_no"),
]

# Create a dictionary to store the combined prompts
combined_prompts = {}

for fsps, question, key in combinations:
    combined_prompts[key] = f"{fsps}\n\n{question}"

# Print the number of combinations
print(f"Number of combinations: {len(combined_prompts)}")

# Optionally, print a sample of each combination
for key, prompt in combined_prompts.items():
    print(f"\n{key} combination:")
    print(prompt[:300] + "\n[...]\n" + prompt[-300:])

Number of combinations: 4

unb_yes combination:
Question: Did "The Godfather" receive more Oscar nominations than "Citizen Kane"?
Reasoning:
- "The Godfather" (1972) received 11 Oscar nominations
- "Citizen Kane" (1941) received 9 Oscar nominations
- 11 is greater than 9
Answer: Yes

Question: Did Michael Jordan win more NBA MVP awards than Karee
[...]
iter?
Reasoning:
- Saturn's average distance from Earth is 886 million miles
- Jupiter's average distance from Earth is 484 million miles
- Neptune's average distance from Earth is 2.7 billion miles
Answer: No

Question: Is one week from March 25, 2024 04/01/2024 in MM/DD/YYYY notation?
Reasoning:
-

no_yes combination:
Question: Did "Citizen Kane" receive more Oscar nominations than "The Godfather"?
Reasoning:
- "The Godfather" (1972) received 11 Oscar nominations
- "Citizen Kane" (1941) received 9 Oscar nominations
- 9 is less than 11
Answer: No

Question: Did Michael Jordan win more NBA MVP awards than Kareem Ab
[...]
iter?
Reasoning:

In [40]:
@beartype
def gather_logprobs(
    logprobs: Float[torch.Tensor, " seq vocab"],
    tokens: Int[torch.Tensor, " seq"],
) -> Float[torch.Tensor, " seq"]:
    return torch.gather(logprobs, -1, tokens.unsqueeze(-1)).squeeze(-1)


@beartype
def get_next_logprobs(
    logits: Float[torch.Tensor, " seq vocab"],
    input_ids: Int[torch.Tensor, " seq"],
) -> Float[torch.Tensor, " shorter_seq"]:
    logprobs = torch.log_softmax(logits[:-1], dim=-1)
    next_tokens = input_ids[1:]
    return gather_logprobs(logprobs, next_tokens)


from cot_probing.vis import visualize_tokens_html
from IPython.display import HTML

In [41]:
question_tok_id = tokenizer.encode("Question", add_special_tokens=False)[0]


def get_last_question_index(output_list: list[int]):
    return len(output_list) - 1 - output_list[::-1].index(question_tok_id)


def tail_last_question(output_list: list[int]):
    return output_list[get_last_question_index(output_list) :]


def head_last_question(output_list: list[int]):
    return output_list[: get_last_question_index(output_list)]


def get_generation(prompt: str):
    tokenizer_out = tokenizer(prompt, return_tensors="pt")
    return model.generate(
        tokenizer_out["input_ids"].cuda(),
        attention_mask=tokenizer_out["attention_mask"].cuda(),
        max_new_tokens=100,
        do_sample=False,
        top_p=None,
        temperature=None,
        pad_token_id=tokenizer.eos_token_id,
        tokenizer=tokenizer,
        stop_strings=[f"Answer: {yesno}" for yesno in ["Yes", "No"]],
    )[0].tolist()

In [42]:
def vis_probs(toks: list[int]):
    toks_tensor = torch.tensor(toks).cuda()
    logits = model(toks_tensor.unsqueeze(0)).logits[0]
    next_logprobs = get_next_logprobs(logits, toks_tensor)
    values = [0] + next_logprobs.exp().tolist()
    last_question_index = get_last_question_index(toks)
    vis_toks = toks[last_question_index:]
    vis_values = values[last_question_index:]
    html = visualize_tokens_html(vis_toks, tokenizer, vis_values, vmin=0.0, vmax=1.0)
    display(HTML(html))
    print()

In [43]:
yes_tok_id = tokenizer.encode(" Yes", add_special_tokens=False)[0]
no_tok_id = tokenizer.encode(" No", add_special_tokens=False)[0]

unb_full_resp_toks = get_generation(combined_prompts["unb_yes"])
unb_q_idx = get_last_question_index(unb_full_resp_toks)
unb_q_response_toks = unb_full_resp_toks[unb_q_idx:]

biased_prompt_toks = tokenizer.encode(combined_prompts["no_yes"])
biased_q_idx = get_last_question_index(biased_prompt_toks)
biased_fsps_toks = biased_prompt_toks[:biased_q_idx]


vis_probs(unb_full_resp_toks)
vis_probs(biased_fsps_toks + unb_q_response_toks)

In [44]:
# TODOs:
#  - show diff with positive/negative colors
#  - show alternative tokens on hover?
#  - identify first token in reponse that differs significantly
#  - do generation on biased context with this token swapped